В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние `25%` объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
from sklearn.datasets import load_boston

boston = load_boston()

X, y = boston.data, boston.target

In [2]:
from sklearn.cross_validation import train_test_split

s = int(len(X) * .75)

X_train, X_test = X[:s], X[s:]
y_train, y_test = y[:s], y[s:]

#train_test_split(X, y, test_size=0.75)

### Задание 1

Как вы уже знаете из лекций, бустинг - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки. Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа y на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

$L(y, z) = (z - y) ^ 2$

$L'(y, z) = 2 \cdot (z - y)$


### Задание 2

Заведите массив для объектов DecisionTreeRegressor (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). В цикле от обучите последовательно 50 решающих деревьев с параметрами max_depth=5 и random_state=42 (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке X:

```python
def gbm_predict(X):
    return [
        sum([coeff * algo.predict([x])[0] for algo, coeff in zip(
            base_algorithms_list, coefficients_list)]) for x in X
    ]
```

(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью mean_squared_error в sklearn.metrics. Возведите результат в степень 0.5, чтобы получить RMSE. Полученное значение RMSE — ответ в пункте 2.

In [3]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=5, random_state=42).fit(X_train, y_train)

coefficients_list = [0.9, ] * 50
base_algorithms_list = [tree, ]

In [4]:
def gbm_predict(X):
    return [
        sum([coeff * algo.predict([x])[0] for algo, coeff in zip(
            base_algorithms_list, coefficients_list)]) for x in X
    ]

In [5]:
def L(x):
    return y_train - gbm_predict(x)

In [6]:
for i in range(49):
    tree = DecisionTreeRegressor(max_depth=5, random_state=42).fit(X_train, L(X_train))
    base_algorithms_list.append(tree)
    

In [7]:
from sklearn.metrics import mean_squared_error

res = mean_squared_error(y_test, gbm_predict(X_test)) ** 0.5
print res

with open('boost_2.txt', 'w') as resf:
    resf.write(str(res))


5.47665097417


### Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где i - номер итерации (от 0 до 9). Используйте качество работы алгоритма как ответ в пункте 3. В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [8]:
tree = DecisionTreeRegressor(max_depth=5, random_state=42).fit(X_train, y_train)

coefficients_list = [0.9, ]
base_algorithms_list = [tree, ]

for i in range(1, 50):
    tree = DecisionTreeRegressor(max_depth=5, random_state=42).fit(X_train, L(X_train))
    base_algorithms_list.append(tree)
    coefficients_list.append(0.9 / (1.0 + i % 10))


In [9]:
print coefficients_list

[0.9, 0.45, 0.3, 0.225, 0.18, 0.15, 0.1285714285714286, 0.1125, 0.1, 0.09, 0.9, 0.45, 0.3, 0.225, 0.18, 0.15, 0.1285714285714286, 0.1125, 0.1, 0.09, 0.9, 0.45, 0.3, 0.225, 0.18, 0.15, 0.1285714285714286, 0.1125, 0.1, 0.09, 0.9, 0.45, 0.3, 0.225, 0.18, 0.15, 0.1285714285714286, 0.1125, 0.1, 0.09, 0.9, 0.45, 0.3, 0.225, 0.18, 0.15, 0.1285714285714286, 0.1125, 0.1, 0.09]


In [10]:
res = mean_squared_error(y_test, gbm_predict(X_test)) ** 0.5
print res

with open('boost_3.txt', 'w') as resf:
    resf.write(str(res))

4.9284335307


### Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке sklearn, так и в сторонней библиотеке XGBoost, которая имеет свой питоновский интерфейс. На практике XGBoost работает заметно лучше GradientBoostingRegressor из sklearn, но для этого задания вы можете использовать любую реализацию. Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет ответ в п.4):

1. С увеличением числа деревьев начиная с некоторого момента качество работы градиентного бустинга не меняется существенно.
2. С увеличением числа деревьев начиная с некоторого момента градиентный бустинг начинает переобучаться.
3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться
4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться



In [11]:
ans = dict((i, None) for i in range(1, 5))

In [16]:
def test_gb2(n_estimators=50, max_depth=5):
    tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42).fit(X_train, y_train)

    global coefficients_list, base_algorithms_list
    
    coefficients_list = [1, ] * n_estimators
    base_algorithms_list = [tree, ]

    for i in range(1, n_estimators):
        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42).fit(X_train, L(X_train))
        base_algorithms_list.append(tree)

    return mean_squared_error(y_test, gbm_predict(X_test)) ** 0.5


In [17]:
from sklearn.ensemble import GradientBoostingRegressor

def test_gb1(**kwargs):
    gb = GradientBoostingRegressor(**kwargs)
    gb.fit(X_train, y_train)
    return mean_squared_error(y_test, gb.predict(X_test)) ** .5

def test_gb(**kwargs):
    return test_gb2(**kwargs)

In [23]:
for tree_n in (1, 2, 3, 4, 5, 10, 15, 35, 40, 50, ): #80, 90, 100, 200, 300, 500):
    print tree_n, test_gb(n_estimators=tree_n)


1 4.87261044427
2 4.86869238127
3 4.44431403906
4 4.64566790569
5 4.76022327731
10 4.96798718595
15 4.96030999029
35 4.99573294888
40 4.99228893185
50 4.9908867057


In [24]:
ans[1] = 0
ans[2] = 1

In [28]:
for tree_d in (1, 2, 3, 4, 5, 10, 15, 35, 40, 50, ): # 80, 90, 100, 200, 300, 500):
    print tree_d, test_gb(max_depth=tree_n)

1 5.81723613243
2 5.81723613243
3 5.81723613243
4 5.81723613243
5 5.81723613243
10 5.81723613243
15 5.81723613243
35 5.81723613243
40 5.81723613243
50 5.81723613243


In [25]:
ans[3] = 1
ans[4] = 0

In [30]:
with open('boost_4.txt', 'w') as resf:
    resf.write(' '.join((str(key) for key, value in ans.iteritems() if value)))

### Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. Для этого обучите LinearRegression из sklearn.linear_model (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке RMSE. Полученное качество - ответ в пункте 5. В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [33]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [36]:
pred = lr.predict(X_test)
res = mean_squared_error(y_test, pred) ** .5
print res

with open('boost_5.txt', 'w') as resf:
    resf.write(str(res))

8.27046803494
